## Evaluate F1 - score or Confusion matrix by using Fine-Tuning ResNet18

In [1]:
import os 
import time
import numpy as np

from sklearn.metrics import f1_score, confusion_matrix, classification_report
from tqdm import tqdm

import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, ConcatDataset

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

## GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


In [5]:
class CUB_CAPTCHA(Dataset):
    def __init__(self, transform, mode='train'):
        self.transform = transform
        self.mode = mode
        base_path = 'CAPTCHA_IMAGES_SPLIT'  # 기본 경로 설정

        if self.mode == 'train':
            folder_path = os.path.join(base_path, 'train')
        elif self.mode == 'valid':
            folder_path = os.path.join(base_path, 'val')

        # 모든 하위 폴더의 파일을 포함
        self.image_paths = []
        self.labels = []

        for class_name in os.listdir(folder_path):
            class_path = os.path.join(folder_path, class_name)
            if os.path.isdir(class_path):
                for img_name in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_name)
                    if os.path.isfile(img_path):  # 파일만 추가
                        self.image_paths.append(img_path)
                        self.labels.append(class_name)  # 클래스명으로 라벨 설정

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)

        # 라벨을 클래스 이름에서 추출
        label = self.labels[idx]

        # 문자열 라벨을 숫자로 매핑
        label_map = {  # 클래스 이름을 정수로 매핑
            'motorcycle': 0, 'seaplane': 1, 'boat': 2, 'motorbus': 3,
            'bicycle': 4, 'train': 5, 'truck': 6, 'airplane': 7
        }
        label = label_map[label]

        return img, label


In [4]:
'''import splitfolders
splitfolders.ratio("CAPTCHA_DCGAN", output="CAPTCHA_DCGAN_split", seed=1004, ratio=(0, 1))'''

'import splitfolders\nsplitfolders.ratio("CAPTCHA_DCGAN", output="CAPTCHA_DCGAN_split", seed=1004, ratio=(0, 1))'

In [6]:
import os
from PIL import Image
from torch.utils.data import Dataset

class CUB_CAPTCHA_TEST(Dataset):
    def __init__(self, transform, mode='valid'):
        self.transform = transform
        self.mode = mode
        base_path = 'CAPTCHA_DCGAN_split'  # 기본 경로 설정

        # train과 validation에 따라 폴더 경로 설정
        if self.mode == 'train':
            folder_path = os.path.join(base_path, 'train')
        elif self.mode == 'valid':
            folder_path = os.path.join(base_path, 'val')
        else:
            raise ValueError("mode must be 'train' or 'valid'")

        # 모든 하위 폴더의 파일 경로와 라벨 생성
        self.image_paths = []
        self.labels = []

        # 클래스 이름을 정수로 매핑
        self.label_map = {
            'motorcycle': 0, 'seaplane': 1, 'boat': 2, 'motorbus': 3,
            'bicycle': 4, 'train': 5, 'truck': 6, 'airplane': 7
        }

        for class_name in os.listdir(folder_path):
            class_path = os.path.join(folder_path, class_name)
            if os.path.isdir(class_path) and class_name in self.label_map:  # 유효한 클래스 이름만 포함
                for img_name in os.listdir(class_path):
                    img_path = os.path.join(class_path, img_name)
                    if os.path.isfile(img_path):  # 파일만 추가
                        self.image_paths.append(img_path)
                        self.labels.append(class_name)  # 클래스명으로 라벨 설정

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)

        # 라벨을 클래스 이름에서 추출하고 정수로 변환
        label = self.labels[idx]
        label = self.label_map[label]

        return img, label


In [7]:
# 데이터 변환 정의
'''transforms_train = transforms.Compose([
    transforms.Resize((448, 448)),              # 크기 조정
    transforms.RandomRotation(15),              # 무작위 회전
    transforms.RandomResizedCrop(448, scale=(0.8, 1.0)),  # 무작위 크롭
    transforms.RandomHorizontalFlip(p=0.5),     # 좌우 반전
    transforms.ToTensor(),                      # 텐서 변환
])'''

transforms_train = transforms.Compose([
    transforms.Resize((64, 64)),              # 크기 조정
    transforms.ToTensor(),                      # 텐서 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 픽셀 값을 [-1, 1]로 정규화
])

transforms_val = transforms.Compose([
    transforms.Resize((64, 64)),              # 크기 조정
    transforms.ToTensor(),                      # 텐서 변환
])


In [8]:
BATCH_SIZE = 16

original_train_set = CUB_CAPTCHA(mode='train', transform=transforms_train) 
original_val_set = CUB_CAPTCHA(mode='valid', transform=transforms_train)
CAPTCHA_train_set = ConcatDataset([original_train_set, original_val_set])

val_set = CUB_CAPTCHA_TEST(mode = 'valid', transform = transforms_val)

train_loader = DataLoader(CAPTCHA_train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

print('Num of each dataset:', len(CAPTCHA_train_set), len(val_set))
print("Loaded combined dataloader")

Num of each dataset: 4068 352
Loaded combined dataloader


## Pre - trained ResNet Load

In [9]:
## Model / Optimizer ###
EPOCH = 30
lr = 0.001

model = models.resnet18(weights = "DEFAULT")

### Transfer Learning ###
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 8)
model.to(DEVICE)

params_to_update = []
for name,param in model.named_parameters():
    if 'fc' in name:
        print(name)
        print(param.requires_grad)
        param.requires_grad = True
        params_to_update.append(param)
    else:
        param.requires_grad = False

print("Created a learning model and optimizer")

fc.weight
True
fc.bias
True
Created a learning model and optimizer


In [10]:
optimizer = optim.Adam(params_to_update, lr = 0.001)

In [11]:
### Train / Evaluation ###
def train(model, train_loader, optimizer, epoch):
    model.train()
    for i, (image,target) in enumerate (train_loader):
        image, target = image.to(DEVICE), target.to(DEVICE)
        output = model(image)
        optimizer.zero_grad()
        train_loss = F.cross_entropy(output, target).to(DEVICE)
        
        train_loss.backward() 
        optimizer.step()
        
        if i % 10 ==0:
            print(f"Train_Epoch : {epoch} [{i}/{len(train_loader)}]\tLoss: {train_loss.item():.6f}")
    
    return train_loss

def evaluate(model, val_loader):
    model.eval()
    eval_loss = 0
    correct = 0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for i, (image, target) in enumerate(val_loader):
            image, target = image.to(DEVICE), target.to(DEVICE)
            output = model(image)

            eval_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            # Predicted labels
            pred = output.argmax(dim=1)  # 각 배치에서 가장 높은 값의 인덱스를 예측
            all_preds.extend(pred.cpu().numpy())  # GPU → CPU
            all_targets.extend(target.cpu().numpy())
            
            correct += pred.eq(target).sum().item()
    
    eval_loss /= len(val_loader.dataset)
    eval_accuracy = 100 * correct / len(val_loader.dataset)
    
    # Calculate F1-score
    f1 = f1_score(all_targets, all_preds, average='weighted')
    
    # Generate Confusion Matrix
    conf_matrix = confusion_matrix(all_targets, all_preds)

    # Extract unique classes from the dataset
    unique_classes = sorted(set(all_targets + all_preds))
    target_names = [f"Class {cls}" for cls in unique_classes]
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(all_targets, all_preds, target_names=target_names))
    
    print("\nConfusion Matrix:")
    print(conf_matrix)
    
    return eval_loss, eval_accuracy, f1, conf_matrix



In [12]:
import time
import torch.optim as optim
from tqdm import tqdm
import torch

epochs = 30

def train_model():
    start = time.time()
    best = 0
    
    for epoch in tqdm(range(epochs), desc="Epochs"):
        epoch_start = time.time()
        train_loss = train(model, train_loader, optimizer, epoch)
        epoch_end = time.time()
            
        # 각 에포크에 대한 Computation Time, Memory 사용량 기록
        epoch_time = epoch_end - epoch_start
        
        # Test 결과 기록
        test_loss, test_accuracy, test_f1, conf_matrix = evaluate(model, val_loader)
        
        print(f"[Final] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}%, F1-Score: {test_f1:.4f}")
        
        print("Confusion Matrix:")
        print(conf_matrix)
        
        end = time.time()
        elapsed_time = end - start
        print(f"Elapsed Time: {int(elapsed_time / 3600)}h, {int((elapsed_time % 3600) / 60)}m, {int(elapsed_time % 60)}s")

In [13]:
train_model()

Epochs:   0%|          | 0/30 [00:00<?, ?it/s]

Train_Epoch : 0 [0/255]	Loss: 2.324743
Train_Epoch : 0 [10/255]	Loss: 2.162630
Train_Epoch : 0 [20/255]	Loss: 1.598135
Train_Epoch : 0 [30/255]	Loss: 1.740698
Train_Epoch : 0 [40/255]	Loss: 1.184355
Train_Epoch : 0 [50/255]	Loss: 1.162261
Train_Epoch : 0 [60/255]	Loss: 0.829354
Train_Epoch : 0 [70/255]	Loss: 1.004719
Train_Epoch : 0 [80/255]	Loss: 1.092848
Train_Epoch : 0 [90/255]	Loss: 0.961614
Train_Epoch : 0 [100/255]	Loss: 0.478543
Train_Epoch : 0 [110/255]	Loss: 0.932386
Train_Epoch : 0 [120/255]	Loss: 0.541199
Train_Epoch : 0 [130/255]	Loss: 0.343112
Train_Epoch : 0 [140/255]	Loss: 0.429915
Train_Epoch : 0 [150/255]	Loss: 0.864261
Train_Epoch : 0 [160/255]	Loss: 0.753360
Train_Epoch : 0 [170/255]	Loss: 0.512872
Train_Epoch : 0 [180/255]	Loss: 0.593309
Train_Epoch : 0 [190/255]	Loss: 0.643122
Train_Epoch : 0 [200/255]	Loss: 0.477128
Train_Epoch : 0 [210/255]	Loss: 0.601441
Train_Epoch : 0 [220/255]	Loss: 0.796341
Train_Epoch : 0 [230/255]	Loss: 0.495569
Train_Epoch : 0 [240/255]	L

Epochs:   3%|▎         | 1/30 [00:08<04:10,  8.63s/it]


Classification Report:
              precision    recall  f1-score   support

     Class 0       0.93      0.20      0.33        64
     Class 1       0.40      0.44      0.42        32
     Class 2       0.36      0.70      0.47        64
     Class 3       0.86      0.19      0.31        32
     Class 4       0.67      0.69      0.68        32
     Class 5       0.30      0.09      0.14        32
     Class 6       0.49      0.61      0.55        64
     Class 7       0.33      0.50      0.40        32

    accuracy                           0.45       352
   macro avg       0.54      0.43      0.41       352
weighted avg       0.56      0.45      0.42       352


Confusion Matrix:
[[13  5 10  1  8  2 20  5]
 [ 0 14 13  0  0  0  4  1]
 [ 0  4 45  0  0  0  4 11]
 [ 0  1 12  6  1  2  7  3]
 [ 1  0  5  0 22  0  0  4]
 [ 0  2 18  0  0  3  3  6]
 [ 0  3 18  0  1  1 39  2]
 [ 0  6  5  0  1  2  2 16]]
[Final] Test Loss: 1.6756, Accuracy: 44.8864%, F1-Score: 0.4228
Confusion Matrix:
[[13  5

Epochs:   7%|▋         | 2/30 [00:13<02:52,  6.15s/it]

...


Classification Report:
              precision    recall  f1-score   support

     Class 0       0.95      0.28      0.43        64
     Class 1       0.22      0.56      0.32        32
     Class 2       0.31      0.72      0.43        64
     Class 3       0.53      0.31      0.39        32
     Class 4       0.80      0.62      0.70        32
     Class 5       0.40      0.12      0.19        32
     Class 6       0.61      0.27      0.37        64
     Class 7       0.50      0.34      0.41        32

    accuracy                           0.41       352
   macro avg       0.54      0.40      0.41       352
weighted avg       0.56      0.41      0.41       352


Confusion Matrix:
[[18 15 14  1  4  1  9  2]
 [ 0 18 14  0  0  0  0  0]
 [ 0 13 46  0  0  1  0  4]
 [ 0  6 14 10  0  1  0  1]
 [ 0  5  5  0 20  0  1  1]
 [ 0  9 14  1  0  4  1  3]
 [ 1  8 27  7  1  3 17  0]
 [ 0  7 14  0  0  0  0 11]]
[Final] Test Loss: 2.4287, Accuracy: 40.9091%, F1-Score: 0.4077
Confusion Matrix:
[[18 15

Epochs:  97%|█████████▋| 29/30 [02:10<00:05,  5.32s/it]


Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      0.25      0.40        64
     Class 1       0.28      0.59      0.38        32
     Class 2       0.35      0.66      0.45        64
     Class 3       0.47      0.25      0.33        32
     Class 4       0.80      0.75      0.77        32
     Class 5       0.62      0.16      0.25        32
     Class 6       0.55      0.45      0.50        64
     Class 7       0.41      0.50      0.45        32

    accuracy                           0.45       352
   macro avg       0.56      0.45      0.44       352
weighted avg       0.58      0.45      0.44       352


Confusion Matrix:
[[16 11 11  0  6  1 16  3]
 [ 0 19 10  0  0  0  2  1]
 [ 0 10 42  0  0  0  1 11]
 [ 0  7 11  8  0  2  3  1]
 [ 0  3  4  0 24  0  0  1]
 [ 0  8 11  2  0  5  1  5]
 [ 0  3 25  6  0  0 29  1]
 [ 0  7  7  1  0  0  1 16]]
[Final] Test Loss: 2.2013, Accuracy: 45.1705%, F1-Score: 0.4437
Confusion Matrix:
[[16 11

Epochs: 100%|██████████| 30/30 [02:16<00:00,  4.55s/it]


Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      0.28      0.44        64
     Class 1       0.42      0.53      0.47        32
     Class 2       0.38      0.73      0.50        64
     Class 3       0.43      0.41      0.42        32
     Class 4       0.70      0.81      0.75        32
     Class 5       0.28      0.16      0.20        32
     Class 6       0.65      0.34      0.45        64
     Class 7       0.39      0.62      0.48        32

    accuracy                           0.48       352
   macro avg       0.53      0.49      0.46       352
weighted avg       0.57      0.48      0.46       352


Confusion Matrix:
[[18  7 13  1  9  4  9  3]
 [ 0 17 10  0  0  2  1  2]
 [ 0  2 47  0  1  0  1 13]
 [ 0  0 15 13  0  2  0  2]
 [ 0  2  3  0 26  1  0  0]
 [ 0  7  9  2  0  5  1  8]
 [ 0  3 19 12  1  4 22  3]
 [ 0  2  8  2  0  0  0 20]]
[Final] Test Loss: 2.1836, Accuracy: 47.7273%, F1-Score: 0.4639
Confusion Matrix:
[[18  7